## Predicting Deliveries

In this final project, we'll implement a SARIMA model to predict the number of deliveries in South Africa

In [1]:
### Load in libraries and read in data.
### The data is stored in a dataframe called 'df'.

import pandas as pd
import numpy as np

df = pd.read_csv('deliveries.csv')
df.head()

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,C�te d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,C�te d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN


In [6]:
df['Scheduled Delivery Date']

0         2-Jun-06
1        14-Nov-06
2        27-Aug-06
3         1-Sep-06
4        11-Aug-06
           ...    
10319    31-Jul-15
10320    31-Jul-15
10321    31-Aug-15
10322     9-Sep-15
10323    31-Aug-15
Name: Scheduled Delivery Date, Length: 10324, dtype: object

In [8]:
### Part A (20 Pts):


### Convert the values in the column 'Scheduled Delivery Date' into DateTime objects.
### Set the index of the dataframe to the 'Scheduled Delivery Date' column.

# your code here
df['Scheduled Delivery Date'] = pd.to_datetime(df['Scheduled Delivery Date'])
df.set_index('Scheduled Delivery Date', inplace = True)

In [9]:
### Part B (20 Pts)

### Find the number of deliveries scheduled for South Africa in 2010.
### Store it in a variable called "sa".
### Create a varaible named temp_df to store the dataframe only containing values for South Africa.
### Hint: Countries are stored in a column called 'Country' and years are stored in a column called 'year'.

# your code here
temp_df = df[df['Country'] == 'South Africa']
df['year'] = df.index.year
sa = df.groupby('year').count().loc[2010]['ID']

In [10]:
### Part C (20 pts)

### Create a new dataframe with the variable name 'sr' by using the corrsponding values in the 'Line Item Quantity' column.
### Then, add three new colums corresponding to different shift amounts: 2, 10, and 20.
### The index of the 'sr' dataframe should be 'Scheduled Delivery Date' 
### with the columns: ['Line Item Quantity', 'shift_2', 'shift_10', 'shift_20']

# your code here
sr = pd.DataFrame(df['Line Item Quantity'])
sr['shift_2'] = sr['Line Item Quantity'].shift(2)
sr['shift_10'] = sr['Line Item Quantity'].shift(10)
sr['shift_20'] = sr['Line Item Quantity'].shift(20)

In [11]:
### Part D (20 pts)

### Find the autocorrelation value on the first column in sr. Use a lag of 20. 
### Store the value in variable called ac.

# your code here
ac = sr['Line Item Quantity'].autocorr(10)

In [ ]:
import statsmodels.api as sm

### Part E (20 pts)

### Use the SARIMA model to generate predictions from the sr dataframe. 
### Use the model to find the mean predicted value. Store this mean in a variable called 'pred'.


# your code here
mod = sm.tsa.statespace.SARIMAX(sr['Line Item Quantity'],
                                order=(0, 0, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()

pred = results.get_prediction().predicted_mean.mean()

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  ' forecasting.', ValueWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  ' forecasting.', ValueWarning)
